# Fetching data from ArcGIS Online
The [ArcGIS Python API](https://developers.arcgis.com/python/) is a powerful Python package for accessing ArcGIS Online resources. Here, we'll use it to download MJB&A's exit data to a local shapefile.

A useful tutoral for this workbook: https://developers.arcgis.com/labs/python/download-data/

In [1]:
#Import the arcgis GIS module - used to search AGOL for content
from arcgis import GIS

#Import the
import pandas as pd

### 1. Searching for content
The ArcGIS Python API's [gis module](https://developers.arcgis.com/python/guide/the-gis-module/) allows us to interact with ArcGIS Online content. 
* https://developers.arcgis.com/python/guide/accessing-and-creating-content/#Searching-for-content

In [13]:
#Create a gis object
gis = GIS()
#Use the gis object to search content
results = gis.content.search(query='NC exits 2019 owner:lukehellgren1',
                             max_items = 500)
len(results)

4

In [14]:
#Show a list of results
results

[<Item title:"TCI_NC_exits_2019_06_11_v1" type:Feature Layer Collection owner:lukehellgren1>,
 <Item title:"TCI_NC_exits_2019_05_31_v1" type:Feature Layer Collection owner:lukehellgren1>,
 <Item title:"TCI_NC_exits_2019_05_31_20miBuffer" type:Feature Layer Collection owner:lukehellgren1>,
 <Item title:"TCI_NC_exits_2019_05_31_nodata" type:Feature Layer Collection owner:lukehellgren1>]

In [17]:
#Assign the name "item" the first item in the results list; then show the item
item = results[0]
#Reveal the data type of the "item" object
print(type(item))
#Show the item
item

<class 'arcgis.gis.Item'>


<Item title:"TCI_NC_exits_2019_06_11_v1" type:Feature Layer Collection owner:lukehellgren1>

We just create an ArcGIS "[item]()https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#item" object. This item is our link to a specific resource stored on some ArcGIS Online server. We can show a number of properties associated with this item. 

In [18]:
#Show the item's title
item.title

'TCI_NC_exits_2019_06_11_v1'

In [33]:
#Show the type of service this is
item.type

'Feature Service'

In [32]:
#Show the item's unique id
item.id

'591c9dcc30244ddaa6a35d36571b94dd'

In [31]:
#Show where the item is stored, i.e., its URL
print(item.url)

https://services7.arcgis.com/fqNd6NEGNf5qzQdv/arcgis/rest/services/TCI_NC_exits_2019_06_11_v1/FeatureServer


If you open the link above, you'll see the **REST endpoint** of the service which reveals information about the service includng what layers (if any) are associated with the service. Here we see ours has one layer. We can also get this information via Python

In [35]:
#Show the layers associated with the item
item.layers

[<FeatureLayer url:"https://services7.arcgis.com/fqNd6NEGNf5qzQdv/arcgis/rest/services/TCI_NC_exits_2019_06_11_v1/FeatureServer/1">]

In [36]:
#Extract the one (and only) layer into a new variable
layer = item.layers[0]
type(layer)

arcgis.features.layer.FeatureLayer

Now we have a new object - an ArcGIS [FeatureLayer](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featurelayer) object. We are getting closer to the actual data object...

In [41]:
#Extract a list of fields from the layer's properties object
fields = layer.properties.fields
#Print all the field names
for f in fields: print (f.name)

FID
Exit_ID
Longitude
Latitude
County
State
Highway
Type
AADT
Pop_dens
Comm_1mi
Ports_5mi
Ports_10mi
Ports_20mi
Close_nonT


In [43]:
#Get unique values in a given field
layer.get_unique_values('State')

['PA',
 'DE',
 'NH',
 'MD',
 'DC',
 'VA',
 'ME',
 'RI',
 'MA',
 'NY',
 'CT',
 'NJ',
 'VT',
 'NC']

In [50]:
#Query a set of features from the layer
selected_features = layer.query('State = \'NC\'')
type(selected_features)

arcgis.features.feature.FeatureSet

In [51]:
#How many feature have been returned?
len(selected_features)

1178

When we query the layer, it returns a new object, the arcgis [FeatureSet](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featureset). We can convert this to a [Spatially enabled dataframe] which is much like the familiar GeoPandas geodataframe.

In [65]:
sdfFeatures = selected_features.sdf
sdfFeatures.head()

/opt/conda/lib/python3.7/site-packages/IPython/lib/pretty.py:402: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  return _repr_pprint(obj, self, cycle)
/opt/conda/lib/python3.7/site-packages/pandas/io/formats/html.py:606: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  super().render()


,FID,Exit_ID,Longitude,Latitude,County,State,Highway,Type,AADT,Pop_dens,Comm_1mi,Ports_5mi,Ports_10mi,Ports_20mi,Close_nonT,SHAPE
0,7059,US-64_Cherokee_NC_Exit_1,-84.314304,35.019492,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3700,60.34979,1,0,0,0,34.094152,"{""x"": -84.31430368976345, ""y"": 35.019491849669..."
1,7060,US-64_Cherokee_NC_Exit_2,-84.311492,35.014933,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3700,60.34979,1,0,0,0,34.365230,"{""x"": -84.31149165991212, ""y"": 35.014933379571..."
2,7061,US-64_Cherokee_NC_Exit_3,-84.295528,35.013561,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3900,60.34979,0,0,0,0,35.233979,"{""x"": -84.29552778977563, ""y"": 35.013560589649..."
3,7062,US-64_Cherokee_NC_Exit_4,-84.280291,35.012880,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3900,60.34979,0,0,0,0,36.048303,"{""x"": -84.28029090010494, ""y"": 35.012879950349..."
4,7063,US-64_Cherokee_NC_Exit_5,-84.275190,35.017816,Cherokee,NC,U.S. Highway 64,Exit/Intersection,3900,60.34979,0,0,0,0,36.186597,"{""x"": -84.27519011992803, ""y"": 35.017816120417..."


In [70]:
sdfFeatures.spatial.sr

{'wkid': 4326, 'latestWkid': 4326}

In [86]:
#Create a folder to hold the data
import os
os.mkdir("MJBA")
#Export the data to the folder
sdfFeatures.spatial.to_featureclass(location='MJBA/MJBA_exits.shp')

'MJBA/MJBA_exits.shp'

In [87]:
#Zip
from shutil import make_archive
make_archive('MJBA','zip','MJBA')

'/home/jovyan/work/FetchingData/MJBA.zip'

In [64]:
theMap = gis.map('Durham, NC')
sdfFeatures.spatial.plot(theMap)
theMap

MapView(layout=Layout(height='400px', width='100%'))

## Gas Stations
* https://services.nconemap.gov/secure/rest/services/NC1Map_Energy/MapServer/0

In [88]:
from arcgis.features import FeatureLayer

In [89]:
gasFeatures = FeatureLayer('https://services.nconemap.gov/secure/rest/services/NC1Map_Energy/MapServer/0')

In [92]:
for f in gasFeatures.properties.fields: print(f.name)

OBJECTID
OBJECTID_1
STANDARDS_
County
Name
Address
City
Phone
Title
PreWired
Generator
Longitude
Latitude
Gasoline
Diesel
HV_Diesel
Kerosene
NonHwy
ZIP
Type
Shape


In [94]:
sdfGas = gasFeatures.query().sdf
sdfGas.head()

/opt/conda/lib/python3.7/site-packages/IPython/lib/pretty.py:402: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  return _repr_pprint(obj, self, cycle)
/opt/conda/lib/python3.7/site-packages/pandas/io/formats/html.py:606: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  super().render()


,OBJECTID,OBJECTID_1,STANDARDS_,County,Name,Address,City,Phone,Title,PreWired,...,Longitude,Latitude,Gasoline,Diesel,HV_Diesel,Kerosene,NonHwy,ZIP,Type,SHAPE
0,1,1,19433.0,Macon,4th Street Market,545 North 4th Street,Highlands,8285264191,Unknown,,...,-83.196918,35.055964,24.0,4.0,0.0,0.0,0.0,28741,Commercial,"{""x"": 226894.5777054903, ""y"": 152960.292310301..."
1,2,2,19608.0,Mecklenburg,7-Eleven #30348,801 Wendover Rd.,Charlotte,7043642005,Convenience Store,,...,-80.807186,35.185925,36.0,4.0,0.0,0.0,0.0,28211,Commercial,"{""x"": 445031.08852262166, ""y"": 160783.56405997..."
2,3,3,19212.0,Mecklenburg,7-Eleven #35556,2601 South Blvd.,Charlotte,7045238304,Unknown,,...,-80.865223,35.202306,18.0,2.0,0.0,0.0,0.0,28209,Commercial,"{""x"": 439780.7454674894, ""y"": 162698.089711206..."
3,4,4,17263.0,Mecklenburg,7-Eleven #35577,2519 Beatties Ford Rd.,Charlotte,7049100759,Convenience Store,,...,-80.854070,35.268681,24.0,4.0,0.0,1.0,0.0,28216,Commercial,"{""x"": 440933.4917083445, ""y"": 170040.779521932..."
4,5,5,18665.0,Mecklenburg,7-Eleven #35588,10023 N. Tryon St.,Charlotte,7047902590,Unknown,,...,-80.733719,35.321798,44.0,3.0,0.0,0.0,0.0,28262,Commercial,"{""x"": 451983.8742171372, ""y"": 175734.414666395..."


In [98]:
os.mkdir('GasStations')
sdfGas.spatial.to_featureclass(location='GasStations/GasStations.shp')
make_archive('GasStations','zip','GasStations')

'/home/jovyan/work/FetchingData/GasStations.zip'

In [96]:
sdfGas.spatial.plot(theMap)

True